### PART 02. 프롬프트와 출력 파서

#### Chapter5. 프롬프트

#### 1. 프롬프트 탬플릿 만들기

##### - 프롬프트 탬플릿의 구조
- 지시 사항(Instruction), 질문(Question), 문맥(Context)로 구성

##### - PromptTemplate 객체
- 프롬프트 템플릿 객체

- `template` : 프롬프트 문자열(지시사항, 질문, 문맥 등 지정)

- `input_variables` : 입력해야할 변수 목록(명), LangChain이 자동 추출(from_template()로)

- `partial_variables` : 사전 입력해둔 변수 목록(명)

##### - 프롬프트 템플릿 만드는 방법
방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

- 치환될 변수를 `placeholder`로 생성하여 `{ }`로 묶어서 넣을 문자열 정의

In [ ]:
from dotenv import load_dotenv

load_dotenv()

# LangSmith 추적을 설정합니다. https://smith.langchain.com
#!pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

`template` 매개변수에 값을 치환하여 생성(렌더링)하는 방법

- format 메소드 사용(수동 렌더링)

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?" # country는 단순 변수명

# from_template 메소드로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# 단순 prompt 생성, format 메소드로 변수에 값을 치환
# 문자열 프롬프트 반환(PromptTemplate 객체 X)
prompt = prompt.format(country="대한민국")

- chain 생성 후 invoke 메소드 사용(자동 렌더링)
 : PromptTemplate 내부에서 format 메소드 실행
 → LangChain이 자동 실행하여 자동 렌더링된다고 함

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?" # country는 단순 변수명

# from_template 메소드로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
# ChatOpenAI는 메시지 객체(AIMessage)를 반환하여 content 속성으로 접근
chain.invoke("대한민국").content


방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
- `template`과 `input_variables` 매개변수 직접 지정

- `template`이 받는 인자가 1개인 경우

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"], # input_variables값을 직접 지정, country 변수가 아닐 경우 error 발생
)

# prompt 생성
prompt.format(country="대한민국")

- `template`이 받는 인자가 여러개인 경우

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # country2 값 미리 지정
    },
)

prompt.format(country1="대한민국")
prompt_partial = prompt.partial(country2="캐나다") # partial 메소드로 기존 PromptTemplate 객체를 복사하여 country2 값 변경
prompt_partial.format(country1="대한민국")

chain = prompt_partial | llm

chain.invoke("대한민국").content
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

##### - 부분 변수 활용하기
: 사용자 정의 함수를 통해 프롬프트의 일부를 미리 설정 ex) 현재 날짜 자동 출력



In [ ]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # today 함수 미리 지정
    },
)

# prompt 생성
prompt.format(n=3)

# chain 생성
chain = prompt | llm

print(chain.invoke(3).content)
print(chain.invoke({"today": "Jan 02", "n": 3}).content) # 각 매개변수 값을 직접 지정(today 함수 무시)
# ChatModel(ChatOpenAI) 사용 시 반환되는 AIMessage 객체에서 content로 텍스트를 추출

##### - YAML 파일로 프롬프트 템플릿 로드하기

YAML(YAML Ain’t Markup Language)
 : 

 → load_prompt()로 YAML 파일 불러옴 

In [ ]:
from langchain_core.prompts import load_prompt
from langchain_teddynote.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt.format(fruit="사과") # 프롬프트에 값 치환

prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

#### 2. ChatPromptTemplate

##### - ChatPromptTemplate 정의
: 여러 메시지를 조합하여 대화형 프롬프트를 정의하는 템플릿 객체
→ 튜플 형식으로 `(role, message)`형태로 구성

##### - Role의 구성요소
- `system` : 시스템 설정 메시지 `SystemMessage`로 표현

- `human` : 사용자 입력 메시지 `HumanMessage`로 표현

- `ai` : AI 응답 메시지 `AIMessage`로 표현

##### - ChatPromptTemplate을 만드는 방법

- 방법 1. 단일 메시지
: `from_template()` 사용

- 방법 2. 여러 역할과 메시지
: `from_messages()` 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt.format(country="대한민국")

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message로 tuple 형태로 정의
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 생성
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)

llm.invoke(messages).content

chain = chat_template | llm # chain 생성
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

#### 3. MessagesPlaceholder

##### - MessagesPlaceholder
: 



In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

`conversation` 대화목록을 추가할 경우 `MessagesPlaceholder` 를 사용할 수 있음

In [ ]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

# chain 생성
chain = chat_prompt | llm | StrOutputParser()

# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

#### 4. FewShotPromptTemplate

##### - FewShotPromptTemplate

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성
    model_name="gpt-4.1",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
answer = llm.stream(question)
stream_response(answer)

from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# chain 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)